## How can we get fully-sized images from automated Google searches?

In [1]:
import requests

This code seems to be outdated and just returns empty lists: https://gist.github.com/genekogan/ebd77196e4bf0705db51f86431099e57

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import html

In [3]:
r = requests.get('https://www.google.com/search?q=bunny&source=lnms&tbm=isch&sa=X&ved=2ahUKEwie44_AnqLpAhUhBWMBHUFGD90Q_AUoAXoECBUQAw&biw=1920&bih=947')

In [4]:
soup = BeautifulSoup(r.text)

In [5]:
def find_by_xpath(element_source,xpath_expression):
    root = html.fromstring(element_source)
    return root.xpath(xpath_expression)

In [6]:
find_by_xpath(r.text, '//body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img')

[]

In [7]:
dom_img = soup.find_all('img')[2]
dom_img.parent.parent.parent.parent.parent

<table class="TxbwNb"><tr><td><a href="/url?q=https://www.rd.com/list/cutest-bunnies/&amp;sa=U&amp;ved=2ahUKEwjkrpGD_rrvAhUdJjQIHY53AQ0QqoUBMAF6BAgPEAE&amp;usg=AOvVaw26r-8XU_H8nlU2JwV-6tgR"><div class="RAyV4b"><img alt="" class="t0fcAb" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3ToLkgTspWAGrSpBX3OKSsv9snbS4P0kfr6TDsxNdlXsVUsc6UcnmqN4zSg&amp;s"/></div></a></td></tr><tr><td><a href="/url?q=https://www.rd.com/list/cutest-bunnies/&amp;sa=U&amp;ved=2ahUKEwjkrpGD_rrvAhUdJjQIHY53AQ0Qr4kDMAF6BAgPEAI&amp;usg=AOvVaw3NfylQQlrgRC2UcjxNocVN"><div class="Tor4Ec"> <span class="qXLe6d x3G5ab"> <span class="fYyStc">Cutest Bunnies You'll Want...</span> </span> <span class="qXLe6d F9iS2e"> <span class="fYyStc">rd.com</span> </span> </div></a></td></tr></table>

In [8]:
from ipyplot import plot_images

In [9]:
plot_images([img.get('src') for img in soup.find_all('img')][:5])

In [10]:
plot_images(['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfm7rYp1QfEx9mBR--UKWduDyF_7UdWw7kfxJUUrqvk6fKNnO7IaqwHvGoWA',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3ToLkgTspWAGrSpBX3OKSsv9snbS4P0kfr6TDsxNdlXsVUsc6UcnmqN4zSg',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRLCdV99lxbkf63GKDF1g6Rw-Q4QnzKZx6vTLY7pOpRRBEO09e5WNAP8plL8Q',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjkCo0LSNFO-BAZ2cYDGlEyhVqdVwDWv9wI0z34i7qV96bAmLG69s5fGutEw',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRDToo0Lja0ROGlOG7PreU2eO-kDiZoM-miTAo8K9FlYAWrRNIJPKt7oGPFlm4'])

## Reverse image search with POST request

The idea here is that maybe we should just scrape the thumbnails from google, then later another script could browse through images for which we only have a thumbnail and do a reverse image search with them to find the full-sized image.

In [11]:
# copied from https://stackoverflow.com/questions/23270175/google-reverse-image-search-using-post-request
import requests
import webbrowser

filePath = './thumbnails/0dde0e78a81038ab8482adc1cb94a590.jpg'
searchUrl = 'http://www.google.com/searchbyimage/upload'
multipart = {'encoded_image': (filePath, open(filePath, 'rb')), 'image_content': ''}
response = requests.post(searchUrl, files=multipart, allow_redirects=False)
fetchUrl = response.headers['Location']
webbrowser.open(fetchUrl)

True

Currently this opens a web browser, which won't work on a server. We'll have to set up a headless browser and have it click through until it can download the larger image size.